In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Users\j

True

## Reading File and creating DataFrame

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()


In [3]:
import pandas as pd
wiki_files = pd.read_csv('wiki_sample.csv')
wiki_dataframe = pd.DataFrame(wiki_files)
wiki_dataframe

,content,title,id
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1
1,Anthony United States Post Office\r\n\r\nThe A...,Anthony United States Post Office,2
2,Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...,Dumraon Vidhan Sabha constituency,3
3,Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...,Chaker Khazaal,4
4,Vicente Pascual Pastor\r\n\r\nVicente Pascual ...,Vicente Pascual Pastor,5
...,...,...,...
1286259,Toppr\r\n\r\nToppr is a product of Haygot Educ...,Toppr,1286260
1286260,Outline of education\r\n\r\nThe following outl...,Outline of education,1286261
1286261,Raell Padamsee\r\n\r\nRaell Padamsee is an Ind...,Raell Padamsee,1286262
1286262,Megan Callaway\r\n\r\nMegan Callaway is a foun...,Megan Callaway,1286263


## Preprocessing and Creating Vocab

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

In [ ]:
vocab = list()
term_doc_index = {}
heaps_law_dataset = list()

def apply_function(row, lenTitle, doc_id):
    unique_words = 0
    i = 0
    filtered_content = []
    for word in nltk.word_tokenize(row['content'][lenTitle:]):
        i += 1
        word = lemmatizer.lemmatize(word).lower()
        if word not in stop_words and word.isalnum():
            filtered_content.append(word)
            if word not in vocab:
                unique_words += 1
                vocab.append(word)
            if i % 10000 == 0:
                heaps_law_dataset.append((doc_id, unique_words))
        # if doc_id % 100 == 0:
        #     slope, intercept = heaps_law(heaps_law_dataset, doc_id - 100, doc_id)
        #     print(slope)
        #     if slope < 0.01:
        #         break
    return filtered_content

In [15]:
wiki_dataframe['tokenized_content'] = wiki_dataframe.progress_apply(lambda row: apply_function(row, len(row['title']), row['id']), axis=1)

  3%|▎         | 32693/1286264 [26:06<16:41:13, 20.87it/s]


KeyboardInterrupt: 

In [ ]:
wiki_dataframe.to_pickle('./wiki_dataframe')


In [26]:
import pickle5 as pickle
with open("./wiki_dataframe", "rb") as pickle_file:
  wiki_dataframe_pickle = pickle.load(pickle_file)
# wiki_dataframe_pickle = pd.read_pickle('./wiki_dataframe')

In [27]:
wiki_dataframe_pickle

,content,title,id,tokenized_content
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1,"[arabian, relation, refers, current, historica..."
1,Anthony United States Post Office\r\n\r\nThe A...,Anthony United States Post Office,2,"[anthony, united, states, post, office, listed..."
2,Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...,Dumraon Vidhan Sabha constituency,3,"[dumraon, vidhan, sabha, constituency, one, 24..."
3,Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...,Chaker Khazaal,4,"[chaker, khazaal, born, 28, september, 1987, b..."
4,Vicente Pascual Pastor\r\n\r\nVicente Pascual ...,Vicente Pascual Pastor,5,"[vicente, pascual, pastor, alcoy, june, 3, 186..."
...,...,...,...,...
1286259,Toppr\r\n\r\nToppr is a product of Haygot Educ...,Toppr,1286260,"[toppr, product, haygot, education, private, l..."
1286260,Outline of education\r\n\r\nThe following outl...,Outline of education,1286261,"[following, outline, provided, overview, topic..."
1286261,Raell Padamsee\r\n\r\nRaell Padamsee is an Ind...,Raell Padamsee,1286262,"[raell, padamsee, indian, theatre, personality..."
1286262,Megan Callaway\r\n\r\nMegan Callaway is a foun...,Megan Callaway,1286263,"[megan, callaway, founder, archer, school, gir..."


In [ ]:
vocab = list()
heaps_law_dataset = list()

i = 0
total_words = 0
count = 0
past_unique_words = 0
total_unique_words = 0
for document in tqdm(wiki_dataframe_pickle['tokenized_content']):
    print( total_unique_words - past_unique_words)
    past_unique_words = total_unique_words
    i += 1
    for word in document:
        total_words += 1
        if word not in vocab:
            total_unique_words += 1
            vocab.append(word)
#         if i % 1000 == 0:
#             if count >= 900:
#                 break
#             else:
#                 count = 0
#             heaps_law_dataset.append((total_words, total_unique_words))
        if total_unique_words - past_unique_words < 20:
            count += 1
        else: 
            count = 0

    else:
        continue

    break

  0%|          | 105/1286264 [00:00<21:16, 1007.64it/s]

0
100
44
34
142
75
9
8
111
16
95
29
79
69
28
14
8
2
106
43
24
3
81
22
11
127
24
4
28
29
11
41
4
5
31
26
12
23
53
5
22
29
32
26
9
54
2
17
1
81
8
4
14
13
15
5
11
60
61
15
60
3
37
10
11
12
35
2
20
28
39
60
22
7
29
31
5
60
64
9
28
28
5
16
4
7
16
2
69
71
38
32
2
36
6
24
24
27
26
43
0
1
1
5
11
100
8
35
9
4
5
14
24
3
2
5
16
2
18
9
101
4
18
3
86
3
14
2
1
3
22
4
3
14
2
73
1
34
56
5
2
83
50
11
5
42
2
45
11
14
6
32
11
21
6
89

  0%|          | 206/1286264 [00:00<34:30, 621.12it/s] 


54
13
18
1
16
2
64
91
2
13
1
76
57
52
10
12
17
15
71
1
25
64
52
61
14
15
63
7
79
36
5
4
8
4
2
1
13
29
58
48
4
3
37
6
11
20
52
8
4
14
6
53
3
28
5
29
128
18
3
9
7
9
2
14
7
32
12
4
4
42
11
14
5
39
2
5
18
10
10
3
4
5
74
33
9
4
7
44
10
2
107
6
1
6
20
3
12
5
2
2
7
10
23
22
5
2
5
72

  0%|          | 336/1286264 [00:00<46:34, 460.13it/s]

...

  0%|          | 5991/1286264 [00:39<4:26:21, 80.11it/s]


13
1
7
7
1
1
0
1
14
6
0
1
19
11


  0%|          | 6012/1286264 [00:39<4:24:58, 80.53it/s]

1
1
2
1
10
0
11
1
4
0
0
12
1
1
5
21

  0%|          | 6021/1286264 [00:39<5:42:40, 62.27it/s]

In [54]:
i

1000

In [47]:
len(wiki_dataframe_pickle['tokenized_content'][0])

145

## Heaps Law

In [9]:
import matplotlib.pyplot as plt
import math

def heaps_law(list_to_graph):
        x = list()
        y = list()
        
        for item in list_to_graph:
            x.append(item[0])
            y.append(item[1])
        
        expected_x = list()
        expected_y = list()
        
        k = 20
        for n in range(len(x)):
            expected_x.append(n)
            expected_y.append(math.pow(n, 0.5) * k)

        plt.loglog(x, y)
        plt.loglog(expected_x,expected_y)
        plt.savefig("heaps_law.png")

In [ ]:
inverted_index = pd.DataFrame()
